### Interactive E-commerce dashboard is essential tool for business owner to play around with data and key indicator in a visualizable way. I am using this notebook to demonstrate how to build a dashboard using with Dash Plotly and Bootstrap components.

This dashboard involves basic concept of front-end and back-end HTML web development to enable the filter, date selection feature etc in the graphs. <br />
References: <br />
- Data source [link](https://www.kaggle.com/datasets/mervemenekse/ecommerce-dataset?resource=download)
- Dash component - Cards [link](https://github.com/DashBookProject/Plotly-Dash/blob/master/Bonus-Content/Components/cards.md)
- Dash Document [link](https://dash.plotly.com/)
- Course material [link](https://www.udemy.com/course/python-interactive-dashboards-with-plotly-dash/)

In [1]:
import pandas as pd
#-----------------------------------
# Import data
#-----------------------------------
df = pd.read_csv('E-commerce_Dataset.csv') 

In [2]:
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime
import src.data as data
#import os 
#print(os.path.expanduser('~'))


#-----------------------------------
# Drop rows containing null
#-----------------------------------
df = df.dropna(axis=0)  

# drop undefined payment method
# df = df.drop(df[(df['Payment_method']== 'not_defined')].index)


#-----------------------------------
# Convert the 'Date' column to datetime format
#-----------------------------------
df['Order_Date']= pd.to_datetime(df['Order_Date'])
df.insert(2,'Week',df['Order_Date'].dt.isocalendar().week) #para: (loc, column, value)
df.insert(3, 'Monthly', df['Order_Date'].dt.to_period('M')) # Use datetime.to_period() method to extract Monthly and year
df['Monthly'] = df['Monthly'].astype(str) # convert to string


# Change week to weekly sunday
init_date=datetime.date(2018, 1, 7)
df['Weekly']= [ init_date + datetime.timedelta(days=7*(int(i)-1)) for i in df['Week']] #timedelta accept only int
df['Weekly'] = pd.to_datetime(df['Weekly'])

# Add weekday
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
df['Weekday'] = [weekDays[i.weekday()] for i in df['Order_Date']]


#df['Order_Date'] = df['Order_Date'].dt.date
#df['Weekly'] = df['Weekly'].dt.date
#df['Monthly'] = df['Monthly'].dt.date

#np.float64()


#-----------------------------------
# Change types of customer log-in
#-----------------------------------
# Types of customer log-in
print("before", df.groupby(['Customer_Login_type']).size().index)
# Merge Customer_Login_type
for i in list(df[df['Customer_Login_type'] == 'New '].index):
    df.loc[i,'Customer_Login_type'] = 'First SignUp' 
# Types of customer log-in after merge
print("after", df.groupby(['Customer_Login_type']).size().index)

df.columns = ['Order_Date', 'Time', 'Week', 'Monthly', 'Aging', 'Customer_Id',
       'Gender', 'Device_Type', 'Customer_Login_type', 'Main-category',
       'Sub-category', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping_Cost',
       'Order_Priority', 'Payment_method', 'Weekly','Weekday']





#-----------------------------------
# Add profit margin
#-----------------------------------
df['Profit_margin'] = df['Profit']/df['Sales']

before Index(['First SignUp', 'Guest', 'Member', 'New '], dtype='object', name='Customer_Login_type')
after Index(['First SignUp', 'Guest', 'Member'], dtype='object', name='Customer_Login_type')


In [3]:
#from jupyter_dash import JupyterDash
from dash import Dash, dcc, html 
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output 

#######################
### 1. Build App   ###
#######################

external_stylesheets = [dbc.themes.BOOTSTRAP]# set style, must be compliant with dash_bootstrap_component
app = Dash(__name__, external_stylesheets=external_stylesheets)# create app
#app = JupyterDash(__name__, external_stylesheets=external_stylesheets)# create app

####################
### 2. Font End  ###
####################
# html row1 - dashboard title
header = html.H1(style={'textAlign':'center', 
                        'fontWeight': 'bold', 'color':'#000000', 'padding':'1rem'},
                 children="Ecommerce Earning Dashboard")



# html row2 - data filter
# Designed for time series revenue only
date_panel = html.Div(
    [dbc.Row(html.Label('Date Selection')),
     dbc.Row(dcc.DatePickerRange(id='my-date-picker-range',
                                 display_format='D/M/Y',
                                 min_date_allowed=df['Order_Date'].min().date(),
                                 max_date_allowed=df['Order_Date'].max().date(),
                                 initial_visible_month=df['Order_Date'].min().date(),
                                 start_date=df['Order_Date'].min().date(),
                                 end_date=df['Order_Date'].max().date())),
     dbc.Row(html.Div(id='output-container-date-picker-range'))
    ])
      
    
# html row3 - login/paymet/device filter
# Designed for time series revenue only
top_panel = html.Div(
    [html.Br(), 
     dbc.Row(
         [dbc.Col(html.Label('Multi-Select Login Type'), md=4), 
          dbc.Col(html.Label('Multi-Select Payment Type'), md=4), 
          dbc.Col(html.Label('Multi-Select Device Type'), md=4)
         ]
     ),
     dbc.Row(
         [dbc.Col(dcc.Dropdown(id='login_type',
                               options= [{'label': v, 'value': v} for v in df['Customer_Login_type'].unique()],
                               multi=True,
                               value=df['Customer_Login_type'].unique())),
          dbc.Col(dcc.Dropdown(id='payment_type',
                               options= [{'label': v, 'value': v} for v in df['Payment_method'].unique()],
                               multi=True,
                               value=df['Payment_method'].unique())),
          dbc.Col(dcc.Dropdown(id='device_type',
                               options= [{'label': v, 'value': v} for v in df['Device_Type'].unique()],
                               multi=True,
                               value=df['Device_Type'].unique()))
         ], align="start",
     ),
    ])
       
    
    
# html row4 -  overall earning
# Consist of revenue, profit and aov
# The average order value globally is $109 https://marketing.dynamicyield.com/benchmarks/average-order-value/

card_revenue =  dbc.Card(
    
    dbc.CardBody(
        [
            html.H1("Revenue"),
            html.H3(id="sales_value", className="text-revenue")
        ],
    ),
    className="text-center"
)


card_profit = dbc.Card(
    dbc.CardBody(
        [
            html.H1("Profit"),
            html.H3(id="profit_value", className="text-profit")
        ],
    ),
    className="text-center"
)

# aov formula: total revenue divided by total orders to reflect average amount of money customers spend per order
card_aov =  dbc.Card(
    dbc.CardBody(
        [
            html.H1("AOV"),
            html.H3(id="aov_value", className="text-aov")
        ],
    ),
    className="text-center"
)


    
    

###############################################
### Obseleted but kept for future reference ###
###############################################
# mid_container = html.Div(
#     [html.Br(),
#      dbc.Row(
#          [dbc.Col([html.H4(' Revenue Over Time'),
#                    html.Div([dcc.RadioItems(id='frequency_type',
#                                             options=['Daily', 'Weekly', 'Monthly'],
#                                             value=' Daily', 
#                                             inline = True,
#                                             labelStyle={"display": "flex", "align-items": "center"},
#                                             inputStyle={"margin-left": "50px"}),
#                              dcc.Graph(id='sales_over_time')
#                             ], id="number_over_time")
#                   ], md=6
#                  ),
#           dbc.Col([html.H4('Revenue by Category (Male vs Female)'),
#                    html.Div([dcc.RadioItems(id='product_mainsub',
#                                             options=['Main-category', 'Sub-category'],
#                                             value='Main-category', 
#                                             inline = True,
#                                             labelStyle={"display": "flex", "align-items": "center"},
#                                             inputStyle={"margin-left": "50px"}),
#                             dcc.Graph(id='product_mainsub_view')
#                            ],id="product_cat_card")
#                   ], md=6
#                  )
#          ])
#     ], id="number_over_time_and_product_card")



# html row5 - revenue over time and by gender & category
mid_container0 = html.Div(
    [html.Br(),
     dbc.Row(
         [dbc.Col([html.H4(' Revenue Over Time'),
                   html.Div([dcc.RadioItems(id='frequency_type',
                                            options=['Daily', 'Weekly', 'Monthly'],
                                            value=' Daily', 
                                            inline = True,
                                            labelStyle={"display": "flex", "align-items": "center"},
                                            inputStyle={"margin-left": "50px"}),
                             dcc.Graph(id='sales_over_time')
                            ], id="number_over_time")
                  ], md=5
                 ),
          dbc.Col(md=1),
          dbc.Col([html.H4('Revenue by Category (Male vs Female)'),
                   dcc.Tabs(id="tabs-example-graph", value='Main-category', children=[
                       dcc.Tab(
                           label='Main-category', value='Main-category'),
                       dcc.Tab(
                           label='Sub-category', value='Sub-category', children=[
                               dcc.RadioItems(id='subfilter',
                                              options=['Auto & Accessories', 'Electronic', 'Fashion', 'Home & Furniture'],
                                              value='Auto & Accessories', inline = True,
                                              labelStyle={"display": "flex", "align-items": "center"},
                                              inputStyle={"margin-left": "50px"})
                           ]),
                   ]),
                   dcc.Graph(id='tabs-content-example-graph')
                  ], md=5)
         ])
    ], id="number_over_time_and_product_card")



# html row6 - revenue by category (note: no live filter)
df_login = df.groupby(['Customer_Login_type']).count()['Order_Date'].to_frame()
fig1 = px.pie(df_login, values=list(df_login.Order_Date), names=df_login.index, hole=.3)

df_payment = df.groupby(['Payment_method']).count()['Order_Date'].to_frame()
fig2 = px.pie(df_payment, values=list(df_payment.Order_Date), names=df_payment.index, hole=.3)

df_device = df.groupby(['Device_Type']).count()['Order_Date'].to_frame()
fig3 = px.pie(df_device, values=list(df_device.Order_Date), names=df_device.index, hole=.3)

mid_container_1 = html.Div(id="login_type_card",
                           children = dbc.Row(
                               [dbc.Col([html.H4('Revenue by Log-in Type'),
                                         html.Div(dcc.Graph(id='login_type_view', figure = fig1))], md=4),
                                dbc.Col([html.H4('Revenue by Payment Type'),
                                         html.Div(dcc.Graph(id='payment_type_view', figure = fig2))], md=4),
#                                 dbc.Col([html.H4('Delivery Time (Aging)'),
#                                          html.Div(dcc.Graph(id='aging_view'))
#                                         ], md=8),
                                dbc.Col([html.H4('Revenue by Device Type'), 
                                         html.Div(dcc.Graph(id='paymentn_type_view', figure = fig3))],md=4),
                               ])
                           )



# html row7 - profit margin by sub category
# data preparation
profitmargin_by_main = pd.DataFrame(df.groupby('Main-category').mean()['Profit_margin'].sort_values(ascending=True)).reset_index()
top20_profitmargin = pd.DataFrame(df.groupby('Sub-category').mean()['Profit_margin'].sort_values(ascending=True).tail(20))

# create figures
fig4 = px.pie(profitmargin_by_main, values='Profit_margin', names='Main-category')
fig4.update_traces(textfont_size=20)

fig5 = px.bar(top20_profitmargin, x = 'Profit_margin', y = top20_profitmargin.index, 
              color = 'Profit_margin', color_continuous_scale = ['paleturquoise','blue'], orientation='h')
fig5.update_layout(margin=dict(t=20, b=100, l=50, r=10))

profit_margin_container = html.Div(id="profitmargin_card",
                                  children = dbc.Row(
                                      [dbc.Col([html.H4('Profit Margin by Main-product'), 
                                                html.Div(dcc.Graph(id='profitmargin_main', figure = fig4))], md=4),
                                       dbc.Col([html.H4('Profit Margin by Sub-product (Top 20)'), 
                                                html.Div(dcc.Graph(id='profitmargin_sub', figure = fig5))], md=8)
                                          
                                      ])
                                  )


# consolidate layout
app.layout = html.Div(children=[header, 
                                date_panel, 
                                top_panel, 
                                html.Br(),
                                dbc.Container(
                                    dbc.Row([dbc.Col(card_revenue),
                                             dbc.Col(card_profit),
                                             dbc.Col(card_aov)],
                                           ),fluid=True),
                                #mid_container,
                                mid_container0,
                                mid_container_1, 
                                profit_margin_container], style={"text-align": "left"})


##################
### Back End   ###
##################

# -----sales-----
@app.callback(
    Output('sales_value', 'children'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'),
    Input('payment_type', 'value'),
    Input('login_type', 'value'),
    Input('device_type', 'value')
)


def update_kpi_sales_value(start_date, end_date, payment_type, login_type, device_type):
    dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
    sales = sum(dff['Sales'])
    return f'$ {sales:.2f}'

# -----aov-----
@app.callback(
    Output('aov_value', 'children'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'),
    Input('payment_type', 'value'),
    Input('login_type', 'value'),
    Input('device_type', 'value')
)
def update_kpi_aov_value(start_date, end_date, payment_type, login_type, device_type):
    dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
    aov_value = dff['Sales'].mean()
    return f'$ {aov_value:.2f}'



# -----profit----
@app.callback(
    Output('profit_value', 'children'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'),
    Input('payment_type', 'value'),
    Input('login_type', 'value'),
    Input('device_type', 'value')
)


def update_profit_value(start_date, end_date, payment_type, login_type, device_type):
    dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
    sales = sum(dff['Profit'])
    return f'$ {sales:.2f}'



# -----sales_over_time-----
@app.callback(
    Output('sales_over_time', 'figure'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date'),
    Input('payment_type', 'value'),
    Input('login_type', 'value'),
    Input('device_type', 'value'),
    Input('frequency_type', 'value')
)


def update_sales_over_time(start_date, end_date, payment_type, login_type, device_type, frequency_type):
    dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
    dff = dff.rename({'Order_Date': 'Daily'}, axis=1) 
    dff = dff.groupby(frequency_type.strip()).sum() # use strip() to remove spaces 
    dff = dff['Sales'].to_frame('Sales')
    fig = px.line(dff, x=dff.index, y=list(dff.Sales))
    fig.update_layout(
        #title="Plot Title",
        xaxis_title="Time",
        yaxis_title=" Sales",
        legend_title=" Sales over Time",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        ),
        autosize=True, 
        margin=dict(t=45, b=45, l=50, r=10)
    )
    
    fig.update_xaxes(tickangle=45)
    
    return fig




# -----revenue by gender and product category ----    
@app.callback(Output('tabs-content-example-graph', 'figure'),
              Input('tabs-example-graph', 'value'),
              Input('subfilter', 'value'),
              Input('my-date-picker-range', 'start_date'),
              Input('my-date-picker-range', 'end_date'),
              Input('payment_type', 'value'),
              Input('login_type', 'value'),
              Input('device_type', 'value')
             )
def update_rev_by_gender_product(tab, subfilter, start_date, end_date, payment_type, login_type, device_type):
    
    if tab =='Sub-category':
        dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
        dff = dff[dff['Main-category'] == subfilter]
    elif tab =='Main-category':
        dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
        
    dff_cat = data.product_cat(dff, tab)    
    chart1 = go.Figure()
    
    # Update plot sizing
    chart1.update_layout(
        #width=900,
        height=400,
        autosize=True, margin=dict(t=50, b=0, l=0, r=10),
    )
    chart1.add_trace(go.Bar(x=-dff_cat['Male'].values,
                                y=dff_cat[tab],
                                text=dff_cat["Percent_Male"].map('{:,.0f}%'.format), #Display the numbers with thousands separators in hover-over tooltip 
                                textposition='inside',
                                orientation='h',
                                name='Male',
                                customdata=dff_cat[['Male','Percent_Male']], 
                                hovertemplate = "Category: %{y}<br>Count:%{customdata[0]}<br>Percentage:%{customdata[1]}%<br>Gender:Male<extra></extra>"))
    chart1.add_trace(go.Bar(x= dff_cat['Female'],
                                y =dff_cat[tab],
                                text=dff_cat["Percent_Female"].map('{:,.0f}%'.format), #Display the numbers with thousands separators in hover-over tooltip 
                                textposition='inside',
                                orientation='h',
                                name='Female',
                                customdata=dff_cat[['Female','Percent_Female']], 
                                hovertemplate="Category: %{y}<br>Count:%{x}<br>Percentage:%{customdata[1]}%<br>Gender:Female<extra></extra>")) 
    
    chart1.update_yaxes(categoryorder='array',
                        categoryarray= [dff_cat[tab].sort_values(ascending=False)])
    
    chart1.update_layout(barmode='relative', # align horizontal bars by cat.
                             legend_font_size=15, bargap=0.3,
                             legend_orientation = "h", legend_y=1.15, #legend_x=-0.05,
                             xaxis_title= "Revenue",
                             font=dict(
                                 family="Courier New, monospace",
                                 size=12, color="RebeccaPurple"))
    
    

    return chart1
        
    
# @app.callback(
#     Output('login_type_view', 'figure'),
#     Input('my-date-picker-range', 'start_date'),
#     Input('my-date-picker-range', 'end_date'),
#     Input('payment_type', 'value'),
#     Input('login_type', 'value'),
#     Input('device_type', 'value'),
# ) 
# def update_poduct_by_login_type_chart(start_date, end_date, payment_type, login_type, device_type):
#     dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
#     dff = dff.groupby(['Customer_Login_type']).count()['Order_Date'].to_frame()
#     chart2 = px.pie(dff, values=list(dff.Order_Date), names=dff.index, hole=.3)

#     return chart2  
    

# @app.callback(
#     Output('aging_view', 'figure'),
#     Input('my-date-picker-range', 'start_date'),
#     Input('my-date-picker-range', 'end_date'),
#     Input('payment_type', 'value'),
#     Input('login_type', 'value'),
#     Input('device_type', 'value'),
# ) 
# def update_poduct_by_login_type_chart(start_date, end_date, payment_type, login_type, device_type):
#     dff = data.filter_dataframe(df, start_date, end_date, payment_type, login_type, device_type)
#     dff = dff.groupby(by=['Order_Priority','Aging']).count()['Order_Date']
    
#     df_computed = dff.groupby(level=0).apply(lambda x:100 * x / float(x.sum())).to_frame()
#     df_computed = df_computed.loc[['Critical', 'High', 'Medium', 'Low'],:].reset_index()
#     df_computed.columns = ['Order_Priority', 'Days_to_Deliver', 'Percentage']
    
#     chart3 = px.bar(df_computed, x="Percentage", y="Order_Priority", color='Days_to_Deliver', orientation='h',
#              hover_data=["Days_to_Deliver", "Percentage"], #title='Delivery time',
#              height=400)
    
#     chart3.add_vline(x=25, line_width=3, line_dash="dash", line_color="white")
#     chart3.add_vline(x=50, line_width=3, line_dash="dash", line_color="white")
#     chart3.add_vline(x=75, line_width=3, line_dash="dash", line_color="white")

#     return chart3    









##################
### Run App   ###
#################
if __name__ == "__main__":
    app.run_server(debug = True, use_reloader = False)

                    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
